In [1]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import pandas as pd

dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed'
    }

class Read:
    def __init__(self, start, end):
        self.start = start
        self.end = end

TSS_HK = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1','2','3']:
            TSS_HK.append(TSS(ll[0], int(ll[1])+int(1000)))


TSS_NonGene = []
with open(dic['non_gene'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        TSS_NonGene.append(TSS(ll[0], int(ll[1])+int(1000)))
bamfile = ps.AlignmentFile(dic['bam123'],'rb')

up = 1000
down = 1000

In [24]:
'''
2000*200
raw_data_1 = []
for j, tss in enumerate(TSS_HK):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    distribution_matrix = np.zeros((up+down), 200), dtype=int)
    for r in bamfile.fetch(chrom, start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss,relative_isize] += 1
    raw_data_1.append(distribution_matrix)
raw_data_1 = np.array(raw_data_1)
'''

In [25]:
'''
2000*200
raw_data_0 = []
for j, tss in enumerate(TSS_NonGene[:len(TSS_HK)]):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    distribution_matrix = np.zeros((up+down, 200), dtype=int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss,relative_isize] += 1
    raw_data_0.append(distribution_matrix)
raw_data_0 = np.array(raw_data_0)
'''

In [2]:
'''50*40'''
raw_data_1 = []
for j, tss in enumerate(TSS_HK):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    distribution_matrix = np.zeros((int((up+down)/50), 40), dtype=int)
    for r in bamfile.fetch(chrom, start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//50,relative_isize//5] += 1
    raw_data_1.append(distribution_matrix)
raw_data_1 = np.array(raw_data_1)

In [9]:
'''50*40'''
raw_data_0 = []
for j, tss in enumerate(TSS_NonGene[:len(TSS_HK)]):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    distribution_matrix = np.zeros((int((up+down)/50), 40), dtype=int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//50,relative_isize//5] += 1
    raw_data_0.append(distribution_matrix)
raw_data_0 = np.array(raw_data_0)

In [5]:
data = pd.DataFrame(raw_data_1[1])
data.to_csv('B.csv')

In [38]:
'''
CNN
'''
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import random
data_x = []
data_y = []
for mat in raw_data_1:
    data_x.append(mat)
    data_y.append(1)
for mat in raw_data_0:
    data_x.append(mat)
    data_y.append(0)
perm = random.sample(range(len(data_x)),len(data_x))
data_x = np.array(data_x)
data_y = np.array(data_y)
train_x = data_x[perm[:int(len(perm)*0.8)]]
train_y = data_y[perm[:int(len(perm)*0.8)]]
test_x = data_x[perm[int(len(perm)*0.8):]]
test_y = data_y[perm[int(len(perm)*0.8):]]

In [39]:
train_x = train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2], 1)
# train_y = tf.keras.utils.to_categorical(train_y)
# test_y = tf.keras.utils.to_categorical(test_y)

In [33]:
print(train_x.shape)

(1310, 40, 40, 1)


In [59]:
time_steps = 40
input_size = 40
model = keras.models.Sequential()

model.add(layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(time_steps, input_size,1)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(strides=(2,2),pool_size=(2,2)))
model.add(layers.Conv2D(filters=64,kernel_size=3,strides=(1,1),padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(strides=(2,2),pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(750,activation='relu', kernel_regularizer=None, bias_regularizer=None))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(300,activation='relu',kernel_regularizer=None, bias_regularizer=None))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(16,activation='relu',kernel_regularizer=None, bias_regularizer=None))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=None, bias_regularizer=None))
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(train_x, train_y, epochs=20, batch_size=128, validation_split=0.1,
                        callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./' + 'weight.h5', save_best_only=True)],
                        shuffle=True, verbose=2)
loss, accuracy = model.evaluate(test_x, test_y)
model.save( './cnn.h5')
print(loss, accuracy)
# print(model.summary())

Epoch 1/20
10/10 - 2s - loss: 0.6479 - accuracy: 0.6489 - val_loss: 0.5694 - val_accuracy: 0.8321
Epoch 2/20
10/10 - 1s - loss: 0.5220 - accuracy: 0.7939 - val_loss: 0.4293 - val_accuracy: 0.9084
Epoch 3/20
10/10 - 1s - loss: 0.3887 - accuracy: 0.8685 - val_loss: 0.3397 - val_accuracy: 0.8779
Epoch 4/20
10/10 - 1s - loss: 0.3123 - accuracy: 0.8897 - val_loss: 0.3274 - val_accuracy: 0.8473
Epoch 5/20
10/10 - 1s - loss: 0.2833 - accuracy: 0.8974 - val_loss: 0.2588 - val_accuracy: 0.9160
Epoch 6/20
10/10 - 1s - loss: 0.2415 - accuracy: 0.9177 - val_loss: 0.2428 - val_accuracy: 0.9389
Epoch 7/20
10/10 - 1s - loss: 0.2372 - accuracy: 0.9152 - val_loss: 0.2319 - val_accuracy: 0.9389
Epoch 8/20
10/10 - 1s - loss: 0.2160 - accuracy: 0.9254 - val_loss: 0.2334 - val_accuracy: 0.9160
Epoch 9/20
10/10 - 1s - loss: 0.2023 - accuracy: 0.9304 - val_loss: 0.2215 - val_accuracy: 0.9389
Epoch 10/20
10/10 - 1s - loss: 0.1775 - accuracy: 0.9491 - val_loss: 0.2220 - val_accuracy: 0.9389
Epoch 11/20
10/10 -

In [ ]:
pre = model.predict(test_x,0)
# plt.scatter([x for x in range(len(pre))], pre)
# plt.show()
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_y, pre, pos_label=1)
auc = metrics.auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()
print(thresholds)

In [58]:
from keras import Model
from keras.models import load_model
import numpy as np
import os
import pandas as pd

model = load_model('./cnn.h5')
dense1_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output)
dense1_layer_model.trainable = False
dense1_layer_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30_input (InputLayer) [(None, 40, 40, 1)]       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 40, 40, 32)        320       
_________________________________________________________________
activation_30 (Activation)   (None, 40, 40, 32)        0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 20, 20, 64)        18496     
_________________________________________________________________
activation_31 (Activation)   (None, 20, 20, 64)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 10, 10, 64)        0   